In [ ]:
#imports and data upload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor


sns.set()


df = pd.read_csv("../data/processed/data_for_model_engineered.csv")
df.head()


,Standort_ID,Datum,Koord_Ost,Koord_Nord,Velo,year,day_of_year,Niederschlag,Temperatur Durchschnitt,weekday,...,temp_feels,temp_sq,is_rain,heavy_rain,velo_roll3,velo_roll7,velo_lag1,velo_lag7,station_avg_traffic,location_cluster
0,60,2022-01-01,2682731,1247708,134.0,2022,1,0.0,2.6,5,...,2.60,6.76,0,0,1130.000000,1130.0,1130.0,1130.0,823.141573,0
1,60,2022-01-02,2682731,1247708,189.0,2022,2,0.0,4.9,6,...,4.90,24.01,0,0,1130.000000,1130.0,134.0,1130.0,823.141573,0
2,60,2022-01-03,2682731,1247708,676.0,2022,3,0.0,10.2,0,...,10.20,104.04,0,0,333.000000,1130.0,189.0,1130.0,823.141573,0
3,60,2022-01-04,2682731,1247708,646.0,2022,4,6.4,11.0,1,...,6.52,121.00,1,1,503.666667,1130.0,676.0,1130.0,823.141573,0
4,60,2022-01-05,2682731,1247708,518.0,2022,5,10.9,1.7,2,...,-5.93,2.89,1,1,613.333333,1130.0,646.0,1130.0,823.141573,0


In [4]:
# Definition of feature and target

TARGET = "Velo"

exclude = [
    "Velo",
    "Velo_log",       # nur falls vorhanden
    "Datum",          # rohes Datum nicht direkt ins Modell
    "Standort_ID",
    "Koord_Ost",
    "Koord_Nord",
    "year",
    "week",
    "quarter",
    # Optional: falls du es wirklich nicht brauchst:
    # "day_of_year",
    # "year_length",
]

features = [c for c in df.columns if c not in exclude]
print("Anzahl Features:", len(features))
features

len(features), features[:]


Anzahl Features: 20


(20,
 ['day_of_year',
  'Niederschlag',
  'Temperatur Durchschnitt',
  'weekday',
  'weekend',
  'is_holiday',
  'month',
  'year_length',
  'sin_day',
  'cos_day',
  'temp_feels',
  'temp_sq',
  'is_rain',
  'heavy_rain',
  'velo_roll3',
  'velo_roll7',
  'velo_lag1',
  'velo_lag7',
  'station_avg_traffic',
  'location_cluster'])